# Heston Model Calibration

## Problem Statement
Calibrate the Heston stochastic volatility model to market-observed European option prices.

## Heston Model
The Heston model describes asset price dynamics with stochastic volatility:

$$dS_t = \mu S_t dt + \sqrt{v_t} S_t dW_t^S$$
$$dv_t = \kappa(\theta - v_t)dt + \sigma \sqrt{v_t} dW_t^v$$
$$dW_t^S dW_t^v = \rho dt$$

**Parameters**:
- $v_0$: Initial variance
- $\kappa$: Mean reversion speed
- $\theta$: Long-term variance
- $\sigma$: Volatility of volatility
- $\rho$: Correlation between asset and volatility

## Approach
1. Generate synthetic market data
2. Use characteristic function for fast pricing
3. Minimize least squares error via Levenberg-Marquardt
4. Validate calibrated parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, least_squares
from scipy.integrate import quad
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Heston Characteristic Function

In [ ]:
def heston_characteristic_function(phi, S0, v0, kappa, theta, sigma, rho, T, r):
    """
    Heston characteristic function using the original formulation.
    
    Parameters:
    -----------
    phi : complex
        Characteristic function argument
    S0 : float
        Initial stock price
    v0 : float
        Initial variance
    kappa : float
        Mean reversion speed
    theta : float
        Long-term variance
    sigma : float
        Volatility of volatility
    rho : float
        Correlation between Brownian motions
    T : float
        Time to maturity
    r : float
        Risk-free rate
    
    Returns:
    --------
    complex : Characteristic function value
    """
    # Complex constants
    i = complex(0, 1)
    
    # Intermediate calculations
    d = np.sqrt((rho * sigma * i * phi - kappa)**2 + sigma**2 * (i * phi + phi**2))
    g = (kappa - rho * sigma * i * phi - d) / (kappa - rho * sigma * i * phi + d)
    
    # Characteristic function components
    C = r * i * phi * T + (kappa * theta / sigma**2) * (
        (kappa - rho * sigma * i * phi - d) * T - 2 * np.log((1 - g * np.exp(-d * T)) / (1 - g))
    )
    
    D = ((kappa - rho * sigma * i * phi - d) / sigma**2) * (
        (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    )
    
    return np.exp(C + D * v0 + i * phi * np.log(S0))

## 2. Option Pricing via FFT

In [ ]:
def heston_call_price(K, S0, v0, kappa, theta, sigma, rho, T, r):
    """
    Price European call option using Heston model via Fourier inversion.
    
    Uses the Carr-Madan formula for fast computation.
    """
    # Integration function for P1 (probability ITM under stock measure)
    def integrand_P1(phi):
        return np.real(
            np.exp(-1j * phi * np.log(K)) * 
            heston_characteristic_function(phi - 1j, S0, v0, kappa, theta, sigma, rho, T, r) /
            (1j * phi * heston_characteristic_function(-1j, S0, v0, kappa, theta, sigma, rho, T, r))
        )
    
    # Integration function for P2 (probability ITM under risk-neutral measure)
    def integrand_P2(phi):
        return np.real(
            np.exp(-1j * phi * np.log(K)) * 
            heston_characteristic_function(phi, S0, v0, kappa, theta, sigma, rho, T, r) /
            (1j * phi)
        )
    
    # Compute probabilities via numerical integration
    P1 = 0.5 + (1/np.pi) * quad(integrand_P1, 0, 100)[0]
    P2 = 0.5 + (1/np.pi) * quad(integrand_P2, 0, 100)[0]
    
    # Call price formula
    call_price = S0 * P1 - K * np.exp(-r * T) * P2
    
    return max(call_price, 0)  # Ensure non-negative

## 3. Generate Market Data

In [ ]:
# True Heston parameters (what we'll try to recover)
true_params = {
    'v0': 0.04,      # Initial variance (20% vol)
    'kappa': 2.0,    # Mean reversion speed
    'theta': 0.04,   # Long-term variance
    'sigma': 0.3,    # Vol of vol
    'rho': -0.7      # Correlation (leverage effect)
}

# Market parameters
S0 = 100.0  # Current stock price
r = 0.02    # Risk-free rate
T = 1.0     # Maturity (1 year)

# Generate strike prices (OTM, ATM, ITM)
strikes = np.array([80, 85, 90, 95, 100, 105, 110, 115, 120])

# Generate "market" prices using true parameters
print("Generating synthetic market prices...")
market_prices = np.array([
    heston_call_price(K, S0, true_params['v0'], true_params['kappa'], 
                     true_params['theta'], true_params['sigma'], 
                     true_params['rho'], T, r)
    for K in strikes
])

# Add small noise to simulate market data
np.random.seed(42)
market_prices += np.random.normal(0, 0.05, len(strikes))
market_prices = np.maximum(market_prices, 0.01)  # Ensure positive

print(f"\nMarket Data:")
print(f"{'Strike':<10} {'Market Price':<15}")
print("-" * 25)
for K, price in zip(strikes, market_prices):
    print(f"{K:<10.2f} {price:<15.4f}")

## 4. Calibration via Optimization

In [ ]:
def objective_function(params, strikes, market_prices, S0, T, r):
    """
    Objective function: sum of squared pricing errors.
    
    Returns vector of errors for least_squares optimizer.
    """
    v0, kappa, theta, sigma, rho = params
    
    # Parameter constraints (soft)
    if v0 <= 0 or kappa <= 0 or theta <= 0 or sigma <= 0 or abs(rho) >= 1:
        return np.ones_like(market_prices) * 1e6
    
    # Feller condition: 2*kappa*theta > sigma^2 (ensures positive variance)
    if 2 * kappa * theta <= sigma**2:
        return np.ones_like(market_prices) * 1e6
    
    # Compute model prices
    model_prices = np.array([
        heston_call_price(K, S0, v0, kappa, theta, sigma, rho, T, r)
        for K in strikes
    ])
    
    # Return percentage errors (for better scaling)
    errors = (model_prices - market_prices) / market_prices
    
    return errors

# Initial guess (starting point for optimization)
initial_guess = [0.04, 1.5, 0.04, 0.25, -0.5]

# Parameter bounds
bounds = (
    [0.001, 0.01, 0.001, 0.01, -0.99],  # Lower bounds
    [0.5, 10.0, 0.5, 2.0, 0.99]          # Upper bounds
)

print("Starting calibration...\n")
print(f"Initial guess: v0={initial_guess[0]:.4f}, kappa={initial_guess[1]:.4f}, "
      f"theta={initial_guess[2]:.4f}, sigma={initial_guess[3]:.4f}, rho={initial_guess[4]:.4f}")

# Perform calibration using Levenberg-Marquardt
result = least_squares(
    objective_function,
    initial_guess,
    bounds=bounds,
    args=(strikes, market_prices, S0, T, r),
    method='trf',  # Trust Region Reflective
    ftol=1e-8,
    xtol=1e-8,
    max_nfev=200
)

# Extract calibrated parameters
v0_cal, kappa_cal, theta_cal, sigma_cal, rho_cal = result.x

print(f"\nCalibration completed!")
print(f"\n{'Parameter':<10} {'True Value':<15} {'Calibrated':<15} {'Error %':<10}")
print("-" * 55)
params_names = ['v0', 'kappa', 'theta', 'sigma', 'rho']
true_values = [true_params[p] for p in params_names]
calibrated = result.x

for name, true_val, cal_val in zip(params_names, true_values, calibrated):
    error_pct = abs((cal_val - true_val) / true_val) * 100
    print(f"{name:<10} {true_val:<15.4f} {cal_val:<15.4f} {error_pct:<10.2f}%")

print(f"\nRMSE: {np.sqrt(np.mean(result.fun**2)):.6f}")
print(f"Iterations: {result.nfev}")

## 5. Validation & Visualization

In [ ]:
# Compute calibrated model prices
calibrated_prices = np.array([
    heston_call_price(K, S0, v0_cal, kappa_cal, theta_cal, sigma_cal, rho_cal, T, r)
    for K in strikes
])

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Price comparison
axes[0].plot(strikes, market_prices, 'o-', label='Market Prices', markersize=8, linewidth=2)
axes[0].plot(strikes, calibrated_prices, 's--', label='Calibrated Model', markersize=8, linewidth=2)
axes[0].set_xlabel('Strike Price', fontsize=12)
axes[0].set_ylabel('Option Price', fontsize=12)
axes[0].set_title('Market vs Calibrated Prices', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Right plot: Pricing errors
errors = calibrated_prices - market_prices
axes[1].bar(strikes, errors, color='steelblue', alpha=0.7, edgecolor='black')
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Strike Price', fontsize=12)
axes[1].set_ylabel('Pricing Error', fontsize=12)
axes[1].set_title('Calibration Errors by Strike', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('heston_calibration_results.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nMax absolute error: ${np.max(np.abs(errors)):.4f}")
print(f"Mean absolute error: ${np.mean(np.abs(errors)):.4f}")

## 6. Implied Volatility Surface

In [ ]:
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """Black-Scholes call price."""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def implied_volatility(price, S, K, T, r, tol=1e-6):
    """Compute implied volatility via Newton-Raphson."""
    sigma = 0.3  # Initial guess
    for _ in range(100):
        bs_price = black_scholes_call(S, K, T, r, sigma)
        vega = S * np.sqrt(T) * norm.pdf((np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T)))
        
        diff = bs_price - price
        if abs(diff) < tol:
            return sigma
        
        if vega < 1e-10:
            break
            
        sigma -= diff / vega
        sigma = max(0.01, min(sigma, 2.0))  # Keep sigma in reasonable range
    
    return sigma

# Compute implied volatilities
market_ivs = [implied_volatility(p, S0, K, T, r) for p, K in zip(market_prices, strikes)]
calibrated_ivs = [implied_volatility(p, S0, K, T, r) for p, K in zip(calibrated_prices, strikes)]

# Plot volatility smile
plt.figure(figsize=(10, 6))
plt.plot(strikes/S0, np.array(market_ivs)*100, 'o-', label='Market IV', 
         markersize=8, linewidth=2)
plt.plot(strikes/S0, np.array(calibrated_ivs)*100, 's--', label='Calibrated IV', 
         markersize=8, linewidth=2)
plt.xlabel('Moneyness (K/S)', fontsize=12)
plt.ylabel('Implied Volatility (%)', fontsize=12)
plt.title('Implied Volatility Smile', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.savefig('volatility_smile.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Key Takeaways

**Calibration Challenges**:
1. **Non-convexity**: Multiple local minima exist
2. **Parameter sensitivity**: Small changes in $\sigma$ or $\rho$ significantly affect prices
3. **Feller condition**: Constraint $2\kappa\theta > \sigma^2$ ensures variance stays positive
4. **Numerical stability**: Integration over unbounded domain requires care

**Performance Optimization**:
- FFT-based pricing methods (Carr-Madan) can be 10-100x faster
- GPU acceleration for parallel strike pricing
- Adjoint algorithmic differentiation (AAD) for gradient computation

**Production Considerations**:
- Regularization to prevent overfitting to noisy data
- Time-dependent parameters for better fit
- Market data cleaning (remove stale quotes, arbitrage violations)
- Real-time recalibration as new quotes arrive